In [106]:
import numpy as np
import pandas as pd
import os
import zipfile
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms, models
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from skimage import io, transform
import torch.utils.data as data_utils
import torchvision.models as models
from tabulate import tabulate
from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import timm
import torch.nn.functional as F
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [107]:
''' from google.colab import files
files.upload() '''

' from google.colab import files\nfiles.upload() '

In [108]:
'''os.makedirs('/root/.kaggle', exist_ok=True)
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!pip install -q kaggle

!kaggle competitions download -c dog-breed-identification
with zipfile.ZipFile("dog-breed-identification.zip", 'r') as zip_ref:
     zip_ref.extractall("dog-breed-identification")'''

'os.makedirs(\'/root/.kaggle\', exist_ok=True)\n!cp kaggle.json /root/.kaggle/\n!chmod 600 /root/.kaggle/kaggle.json\n!pip install -q kaggle\n\n!kaggle competitions download -c dog-breed-identification\nwith zipfile.ZipFile("dog-breed-identification.zip", \'r\') as zip_ref:\n     zip_ref.extractall("dog-breed-identification")'

In [109]:
DATA_PATH = '/kaggle/input/dog-breed-identification'
labels = pd.read_csv(f'{DATA_PATH}/labels.csv')
LE = LabelEncoder()
labels['breed_encoded'] = LE.fit_transform(labels['breed'])

# command option c

# Add numerical labels

train_df, valid_df = train_test_split(
    labels,
    train_size=0.8,
    shuffle=True,
    stratify=labels['breed'],
    random_state=42
)

In [110]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [111]:
class DogBreedDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.labels_df = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.labels_df.iloc[idx, 0] + '.jpg')
        image = Image.open(img_name).convert('RGB')
        label = self.labels_df.iloc[idx, -1]

        if self.transform:
            image = self.transform(image)

        return image, label

train_dataset = DogBreedDataset(
    dataframe=train_df,
    root_dir='/kaggle/input/dog-breed-identification/train',
    transform=train_transform
)

valid_dataset = DogBreedDataset(
    dataframe=valid_df,
    root_dir='/kaggle/input/dog-breed-identification/train',
    transform=val_transform
)

batch_size = 64

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4
)

In [112]:
n_classes = 120
batch_size = 64
epochs = 20
lr = 3e-4
gamma = 0.7
seed = 42

In [113]:
'''efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)'''

'efficient_transformer = Linformer(\n    dim=128,\n    seq_len=49+1,  # 7x7 patches + 1 cls-token\n    depth=12,\n    heads=8,\n    k=64\n)'

In [114]:
# List all available models
available_models = timm.list_models()
if "vit_small_patch16_224" in available_models:
    print("yes")

yes


In [115]:
model = timm.create_model('resnet50.a1_in1k', pretrained=True, num_classes=n_classes).to(device)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

In [116]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [117]:
!pip install --upgrade pip
!pip install ipywidgets==8.1.5

In [118]:
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)

        # Forward pass
        output = model(data)
        loss = criterion(output, label)

        # Backward pass and optimization
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        # Calculate accuracy
        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc.item() / len(train_loader)
        epoch_loss += loss.item() / len(train_loader)

    # Step the scheduler
    scheduler.step()

100%|██████████| 128/128 [01:38<00:00,  1.29it/s]


In [119]:
'''from sklearn.model_selection import ParameterGrid


param_grid = {
    'lr': [1e-5, 3e-5, 1e-4],
    'weight_decay': [0.0, 0.01, 0.001],
    'batch_size': [32, 64]
}

best_score = 0
best_params = {}

subset_indices = torch.randperm(len(train_dataset))[:1000]
subset_dataset = torch.utils.data.Subset(train_dataset, subset_indices)

for params in ParameterGrid(param_grid):
    
    model = timm.create_model(
        'resnet50.a1_in1k',
        pretrained=True,
        num_classes=len(LE.classes_),
    ).to(device)
    
    subset_loader = DataLoader(
        subset_dataset,
        batch_size=params['batch_size'],
        shuffle=True
    )
    
    optimizer = optim.AdamW(
        model.parameters(),
        lr=params['lr'],
        weight_decay=params['weight_decay']
    )
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    for epoch in range(3):
        epoch_loss = 0
        epoch_accuracy = 0
        
        for data, label in subset_loader:
            data = data.to(device)
            label = label.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            
            acc = (output.argmax(dim=1) == label).float().mean()
            epoch_accuracy += acc.item() / len(subset_loader)
            epoch_loss += loss.item() / len(subset_loader)
        
        print(f"Epoch {epoch+1} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")
    
    model.eval()
    val_accuracy = 0
    val_loader = DataLoader(valid_dataset, batch_size=64)
    
    with torch.no_grad():
        for data, label in val_loader:
            data = data.to(device)
            label = label.to(device)
            
            output = model(data)
            val_accuracy += (output.argmax(dim=1) == label).float().mean().item() / len(val_loader)
    
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    
    if val_accuracy > best_score:
        best_score = val_accuracy
        best_params = params

print("\nBest parameters:", best_params)
print("Best validation accuracy:", best_score)'''

'from sklearn.model_selection import ParameterGrid\n\n\nparam_grid = {\n    \'lr\': [1e-5, 3e-5, 1e-4],\n    \'weight_decay\': [0.0, 0.01, 0.001],\n    \'batch_size\': [32, 64]\n}\n\nbest_score = 0\nbest_params = {}\n\nsubset_indices = torch.randperm(len(train_dataset))[:1000]\nsubset_dataset = torch.utils.data.Subset(train_dataset, subset_indices)\n\nfor params in ParameterGrid(param_grid):\n    \n    model = timm.create_model(\n        \'resnet50.a1_in1k\',\n        pretrained=True,\n        num_classes=len(LE.classes_),\n    ).to(device)\n    \n    subset_loader = DataLoader(\n        subset_dataset,\n        batch_size=params[\'batch_size\'],\n        shuffle=True\n    )\n    \n    optimizer = optim.AdamW(\n        model.parameters(),\n        lr=params[\'lr\'],\n        weight_decay=params[\'weight_decay\']\n    )\n    criterion = nn.CrossEntropyLoss()\n    \n    model.train()\n    for epoch in range(3):\n        epoch_loss = 0\n        epoch_accuracy = 0\n        \n        for da

In [120]:
'''for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)

        # Forward pass
        output = model(data)
        loss = criterion(output, label)

        # Backward pass and optimization
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        # Calculate accuracy
        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc.item() / len(train_loader)
        epoch_loss += loss.item() / len(train_loader)

    # Step the scheduler
    scheduler.step()'''

'for epoch in range(epochs):\n    model.train()\n    epoch_loss = 0\n    epoch_accuracy = 0\n\n    for data, label in tqdm(train_loader):\n        data = data.to(device)\n        label = label.to(device)\n\n        # Forward pass\n        output = model(data)\n        loss = criterion(output, label)\n\n        # Backward pass and optimization\n        optimizer.zero_grad()  # Zero the gradients\n        loss.backward()  # Backpropagation\n        optimizer.step()  # Update weights\n\n        # Calculate accuracy\n        acc = (output.argmax(dim=1) == label).float().mean()\n        epoch_accuracy += acc.item() / len(train_loader)\n        epoch_loss += loss.item() / len(train_loader)\n\n    # Step the scheduler\n    scheduler.step()'

In [121]:
import torch.nn.functional as F
from PIL import Image
from tqdm import tqdm

def create_submission(model, test_dir, transform, device, breed_names, sample_submission_path):
    # Read the sample submission file to get the correct format
    sample_submission = pd.read_csv(sample_submission_path)
    
    test_files = os.listdir(test_dir)
    model.eval()  # Set the model to evaluation mode

    # Initialize predictions dictionary with zeros
    preds_dict = {img_id: np.zeros(len(breed_names)) for img_id in sample_submission['id']}

    for file in tqdm(test_files, desc="Processing Test Images"):
        img_id = file.split('.')[0]
        if img_id not in preds_dict:
            continue  # Skip files not in the sample submission

        img_path = os.path.join(test_dir, file)
        image = Image.open(img_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():  # Disable gradient calculation
            output = model(image)
            probas = F.softmax(output, dim=1).cpu().numpy()[0]
        
        preds_dict[img_id] = probas

    # Create the submission DataFrame
    submission = sample_submission.copy()
    for i, breed in enumerate(breed_names):
        if breed in submission.columns:
            submission[breed] = submission['id'].map(lambda x: preds_dict[x][i])

    return submission

# Correct paths (example paths - adjust according to your actual dataset)
test_dir = '/kaggle/input/dog-breed-identification/test'  # Should be test directory, not train
sample_submission_path = '/kaggle/input/dog-breed-identification/sample_submission.csv'  # Actual sample submission file

# Generate submission file
# Assuming LE is your LabelEncoder object (should be defined elsewhere in your code)
breed_names = LE.classes_  # Make sure LE is properly defined
submission = create_submission(
    model, 
    test_dir, 
    val_transform, 
    device, 
    breed_names,
    sample_submission_path
)

# Save the submission
output_dir = '/kaggle/working'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'submission.csv')
submission.to_csv(output_path, index=False)

print("Submission shape:", submission.shape)
print('Submission file created successfully!')

Processing Test Images: 100%|██████████| 10357/10357 [02:18<00:00, 74.89it/s]


Submission shape: (10357, 121)
Submission file created successfully!
